## Using Flair Embedding and Glove over English dataset 

In [1]:
import pandas as pd
dataset = pd.read_csv('/home/nesma/SemesterII/Neural Networks/Project/multilingual-text-categorization-dataset/data/dataset.csv', sep='\t', header=None).applymap(str)
dataset.columns = ["Language","label","text"]
languagesData=[]
loc = 0
languages = dataset[dataset.columns[0]].unique()
for i in languages:
    name = languages[loc]+"Data" 
    globals()[name] = pd.DataFrame( dataset[dataset.Language == i])
    loc += 1

In [2]:
len(englishData)
englishData = englishData


In [3]:
def lower_words(text):
    text = text.str.lower()
    return text

englishData['text'] = lower_words(englishData['text'])
input_str = englishData['text']
print(englishData.head())


  Language        label                                               text
0  english  advertising  exclusive: u.s. government seeks facebook help...
1  english  advertising  trump asks sec to mull half-year corporate fil...
2  english  advertising  wall st. up on trade hopes, s&p equals longest...
3  english  advertising  asian shares hit one-year low on turkey, china...
4  english  advertising  asian stocks weaken as turkey worries weigh, d...


In [5]:

import seaborn as sn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
def removes_url(text):
    text = text.apply(lambda x: re.split('https:\/\/.*', str(x))[0])
    return text

def remove_url(text):
    text = text.apply(lambda x: re.split('http:\/\/.*', str(x))[0])
    return text
englishData['text'] = removes_url(englishData['text'])
englishData['text'] = remove_url(englishData['text'])

englishData.head()

,Language,label,text
0,english,advertising,exclusive: u.s. government seeks facebook help...
1,english,advertising,trump asks sec to mull half-year corporate fil...
2,english,advertising,"wall st. up on trade hopes, s&p equals longest..."
3,english,advertising,"asian shares hit one-year low on turkey, china..."
4,english,advertising,"asian stocks weaken as turkey worries weigh, d..."


In [6]:
def remove_numbers(text):
    text = text.str.replace('\d+', '')
    return text


englishData['text'] = remove_numbers(englishData['text'])
englishData.head()

,Language,label,text
0,english,advertising,exclusive: u.s. government seeks facebook help...
1,english,advertising,trump asks sec to mull half-year corporate fil...
2,english,advertising,"wall st. up on trade hopes, s&p equals longest..."
3,english,advertising,"asian shares hit one-year low on turkey, china..."
4,english,advertising,"asian stocks weaken as turkey worries weigh, d..."


In [7]:
def remove_punctuations(text):
    text = text.str.replace('[^\w\s]','')
    return text


englishData['text'] = remove_punctuations(englishData['text'])
englishData.head()

,Language,label,text
0,english,advertising,exclusive us government seeks facebook help to...
1,english,advertising,trump asks sec to mull halfyear corporate fili...
2,english,advertising,wall st up on trade hopes sp equals longest bu...
3,english,advertising,asian shares hit oneyear low on turkey china w...
4,english,advertising,asian stocks weaken as turkey worries weigh do...


In [8]:

def remove_blank_space(col):
    col = col.str.strip()
    col = col.replace('\s+', ' ', regex=True)   
    return col


englishData['text'] = remove_blank_space(englishData.text)
englishData.head()

,Language,label,text
0,english,advertising,exclusive us government seeks facebook help to...
1,english,advertising,trump asks sec to mull halfyear corporate fili...
2,english,advertising,wall st up on trade hopes sp equals longest bu...
3,english,advertising,asian shares hit oneyear low on turkey china w...
4,english,advertising,asian stocks weaken as turkey worries weigh do...


In [9]:
import nltk
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
def remove_stop_words(text, stop):    
    text.apply(lambda x: [item for item in x if item not in stop])
    return text

stop = stopwords.words('english')
remove_stop_words(englishData['text'],stop)
englishData.head()

Using TensorFlow backend.


,Language,label,text
0,english,advertising,exclusive us government seeks facebook help to...
1,english,advertising,trump asks sec to mull halfyear corporate fili...
2,english,advertising,wall st up on trade hopes sp equals longest bu...
3,english,advertising,asian shares hit oneyear low on turkey china w...
4,english,advertising,asian stocks weaken as turkey worries weigh do...


In [10]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [porter_stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)


porter_stemmer = PorterStemmer()
englishData['text'] = englishData['text'].apply(stem_sentences)
englishData.head()

,Language,label,text
0,english,advertising,exclus us govern seek facebook help to wiretap...
1,english,advertising,trump ask sec to mull halfyear corpor file vs ...
2,english,advertising,wall st up on trade hope sp equal longest bull...
3,english,advertising,asian share hit oneyear low on turkey china wo...
4,english,advertising,asian stock weaken as turkey worri weigh dolla...


In [11]:
englishData = englishData[["label","text"]]
englishData['label'] = '__label__' + englishData['label'].astype(str)


In [12]:
englishData.iloc[0:int(len(englishData)*0.8)].to_csv('train.csv', sep='\t', index = False, header = False)
englishData.iloc[int(len(englishData)*0.8):int(len(englishData)*0.9)].to_csv('test.csv', sep='\t', index = False, header = False)
englishData.iloc[int(len(englishData)*0.9):].to_csv('dev.csv', sep='\t', index = False, header = False);

In [13]:
englishData.head()

,label,text
0,__label__advertising,exclus us govern seek facebook help to wiretap...
1,__label__advertising,trump ask sec to mull halfyear corpor file vs ...
2,__label__advertising,wall st up on trade hope sp equal longest bull...
3,__label__advertising,asian share hit oneyear low on turkey china wo...
4,__label__advertising,asian stock weaken as turkey worri weigh dolla...


In [14]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentLSTMEmbeddings,DocumentRNNEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
corpus = NLPTaskDataFetcher.load_classification_corpus(Path('./'), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')


Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
2019-05-11 20:37:53,315 Reading data from .
2019-05-11 20:37:53,316 Train: train.csv
2019-05-11 20:37:53,316 Dev: dev.csv
2019-05-11 20:37:53,317 Test: test.csv


In [15]:
len(corpus.dev)

452

In [16]:
word_embeddings = [WordEmbeddings('glove'),FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]

In [17]:
document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)


In [18]:
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)


In [19]:
trainer = ModelTrainer(classifier, corpus)


In [20]:
trainer.train('./', max_epochs=20, patience=1)

2019-05-11 20:39:12,505 ----------------------------------------------------------------------------------------------------
2019-05-11 20:39:12,507 Evaluation method: MICRO_F1_SCORE
2019-05-11 20:39:12,510 ----------------------------------------------------------------------------------------------------
2019-05-11 20:40:22,795 epoch 1 - iter 0/113 - loss 0.11279526
2019-05-11 21:07:42,670 epoch 1 - iter 11/113 - loss 0.11256755


RuntimeError: $ Torch: not enough memory: you tried to allocate 3GB. Buy new RAM! at /opt/conda/conda-bld/pytorch_1550780889552/work/aten/src/TH/THGeneral.cpp:201